In [18]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM


In [19]:
documents = SimpleDirectoryReader(input_files=["../07_rag_frameworks/02_1_LlamaIndex.pdf"]).load_data()

len(documents)

5

In [20]:
embed_model = HuggingFaceEmbedding(
    model_name="all-MiniLM-L6-v2"
)

2025-12-24 14:00:25,473 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [31]:
llm = HuggingFaceLLM(
    model_name="gpt2",
    tokenizer_name="gpt2",
    max_new_tokens=100,
    generate_kwargs={"do_sample": False, "top_p": 0.9}
)

In [32]:
index = VectorStoreIndex.from_documents(
    documents,
    embed_model=embed_model,
    llm=llm
)

In [33]:
index

In [34]:
query_engine = index.as_query_engine(
    similarity_top_k=3,
    llm=llm
)


In [35]:
question = "what is LlamaIndex?"

# Retrieve relevant documents
retriever = index.as_retriever(similarity_top_k=3)
nodes = retriever.retrieve(question)

print("Top 3 Retrieved Documents:\n")
for i, node in enumerate(nodes, 1):
    print(f"--- Document {i} ---")
    print(node.get_content()[:300])
    print()

Top 3 Retrieved Documents:

--- Document 1 ---
LlamaIndex Basics: An Introduction for
Retrieval-Augmented Generation
1. Introduction to LlamaIndex
LlamaIndexis a framework designed to connect Large Language Models (LLMs) with
external data sources such as PDF documents, text files, databases, and web content.
In essence, LlamaIndex enables an LL

--- Document 2 ---
6. Comparison with LangChain
Aspect LlamaIndex LangChain
Primary focus Data indexing and retrieval Workflow orchestration
Ease of use High (beginner-friendly) Moderate
RAG suitability Very strong Strong
Abstraction level High Medium
Pipeline flexibility Limited Highly flexible
As a general guideline

--- Document 3 ---
10. Recommended Learning Path
A suggested progression for learning LlamaIndex is:
1. Understanding document ingestion and nodes
2. Learning index construction
3. Implementing query mechanisms
4. Building PDF-based question answering systems
5. Integrating a Large Language Model
6. Adding citation an



In [37]:
# Get the best retrieved document
best_node = nodes[0]
print(" Best Retrieved Document:\n")
print(f"Score: {best_node.score}")
print(f"\nContent:\n{best_node.get_content()[:500]}")
print(f"\nMetadata: {best_node.metadata}")


 Best Retrieved Document:

Score: 0.6222384434677171

Content:
LlamaIndex Basics: An Introduction for
Retrieval-Augmented Generation
1. Introduction to LlamaIndex
LlamaIndexis a framework designed to connect Large Language Models (LLMs) with
external data sources such as PDF documents, text files, databases, and web content.
In essence, LlamaIndex enables an LLM to read, search, and generate answers based
on user-provided documents. It is primarily used for buildingRetrieval-Augmented
Generation (RAG)systems.
2. Motivation for Using LlamaIndex
By default, L

Metadata: {'page_label': '1', 'file_name': '02_1_LlamaIndex.pdf', 'file_path': '..\\07_rag_frameworks\\02_1_LlamaIndex.pdf', 'file_type': 'application/pdf', 'file_size': 75694, 'creation_date': '2025-12-24', 'last_modified_date': '2025-12-24'}


In [41]:
from llama_index.core.retrievers import VectorIndexRetriever

# Vector retriever (semantic search)
vector_retriever = VectorIndexRetriever(index=index, similarity_top_k=5)

print("=" * 70)
print("HYBRID SEARCH COMPARISON")
print("=" * 70)

# Vector search results
print("\nVector Search Results:")
print("-" * 70)
vector_nodes = vector_retriever.retrieve(question)
for i, node in enumerate(vector_nodes, 1):
    print(f"\n{i}. Score: {node.score:.4f}")
    print(f"   Content: {node.get_content()[:250]}...")

# Simple keyword-based filtering
print("\n\nKeyword Search Results:")
print("-" * 70)
keywords = question.lower().split()
keyword_matches = []
for doc in documents:
    content = doc.get_content().lower()
    match_count = sum(1 for kw in keywords if kw in content)
    if match_count > 0:
        keyword_matches.append((doc, match_count))

keyword_matches.sort(key=lambda x: x[1], reverse=True)
for i, (doc, matches) in enumerate(keyword_matches[:3], 1):
    print(f"\n{i}. Keyword Matches: {matches}")
    print(f"   Content: {doc.get_content()[:250]}...")

print("\n\nHybrid approach combines semantic similarity with keyword matching!")


HYBRID SEARCH COMPARISON

Vector Search Results:
----------------------------------------------------------------------

1. Score: 0.6222
   Content: LlamaIndex Basics: An Introduction for
Retrieval-Augmented Generation
1. Introduction to LlamaIndex
LlamaIndexis a framework designed to connect Large Language Models (LLMs) with
external data sources such as PDF documents, text files, databases, and...

2. Score: 0.5284
   Content: 6. Comparison with LangChain
Aspect LlamaIndex LangChain
Primary focus Data indexing and retrieval Workflow orchestration
Ease of use High (beginner-friendly) Moderate
RAG suitability Very strong Strong
Abstraction level High Medium
Pipeline flexibil...

3. Score: 0.5014
   Content: 10. Recommended Learning Path
A suggested progression for learning LlamaIndex is:
1. Understanding document ingestion and nodes
2. Learning index construction
3. Implementing query mechanisms
4. Building PDF-based question answering systems
5. Integr...

4. Score: 0.4812
   Content